# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [576]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML

In [577]:
data = pd.read_csv('labeled.csv')

In [578]:
data

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [579]:
X_train, X_test, y_train, y_test = train_test_split(data.comment.tolist(), data.toxic.tolist(), test_size=0.3, random_state=42)

### Обучим на встроенном токенайзере sklearn

In [580]:
vectorizer = CountVectorizer()

In [581]:
print(f'X_train shape: {len(X_train)}, y_train shape: {len(y_train)},\n'
      f'X_test shape: {len(X_test)}, y_test shape: {len(y_test)}')

X_train shape: 10088, y_train shape: 10088,
X_test shape: 4324, y_test shape: 4324


In [582]:
vectorizer.fit_transform(['Полина любит кошек.', 'И питон и кошек.'])

<2x4 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [583]:
X_train_default = vectorizer.fit_transform(X_train)
X_test_default = vectorizer.transform(X_test)

In [584]:
knn = KNeighborsClassifier(n_neighbors=3)

In [585]:
knn.fit(X_train_default, y_train)

KNeighborsClassifier(n_neighbors=3)

In [586]:
y_pred_default = knn.predict(X_test_default)

### Обучим на токенайзере razdel.tokenize

In [587]:
!pip install razdel

import razdel

In [588]:
def razdel_tokenizer(smth):
  tokens = [i.text for i in list(razdel.tokenize(smth))]
  return tokens

In [589]:
vectorizer = CountVectorizer(tokenizer=razdel_tokenizer,
                             token_pattern=None)

In [590]:
X_train_razdel = vectorizer.fit_transform(X_train)
X_test_razdel = vectorizer.transform(X_test)

In [591]:
knn.fit(X_train_razdel, y_train)

KNeighborsClassifier(n_neighbors=3)

In [592]:
y_pred_razdel = knn.predict(X_test_razdel)

### Сравним результаты

In [593]:
print(classification_report(y_test, y_pred_default, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.79      0.51      0.62      2906
         1.0       0.42      0.72      0.53      1418

    accuracy                           0.58      4324
   macro avg       0.60      0.61      0.57      4324
weighted avg       0.67      0.58      0.59      4324



In [594]:
print(classification_report(y_test, y_pred_razdel, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.76      0.65      0.70      2906
         1.0       0.45      0.58      0.51      1418

    accuracy                           0.63      4324
   macro avg       0.60      0.61      0.60      4324
weighted avg       0.66      0.63      0.64      4324



Классификатор с векторайзером на razdel.tokenize лучше не пропускает токсичные комментарии, см. precision, т.е. для удаления токсичных комментариев лучше использовать его (например, чтобы не "наказывать" за на самом деле нетоксичные комментарии). Хоть и точность не самая лучшая (можно повысить препроцессингом/посмотреть стемминг и тд, но в задании не было таких требований). По recall, если же нам важно отловить как можно больше токсичных комментариев, лучше использовать векторайзер со встроенным токенизатором.
Если важны оба, и нам важно, какой векторайзер "в целом" лучше -- смотрим, где лучше F мера. Таким образом, лучший векторайзер с токенизатором из razdel.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [595]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [596]:
!pip install pymystem3

In [597]:
from pymystem3 import Mystem
import string

In [598]:
mystem = Mystem(disambiguation=True)

In [599]:
def custom_tokenizer(smth):
  smth = smth.translate(str.maketrans('', '', string.punctuation))
  tokens = list(mystem.lemmatize(smth))
  return tokens

CountVectorizer + LogisticRegression

In [600]:
cvec = CountVectorizer(tokenizer=custom_tokenizer,
                       token_pattern=None,
                       stop_words=russian_stopwords,
                       lowercase=True,
                       ngram_range=(1, 2),
                       max_df=1.0,
                       min_df=0.001
                       )

In [601]:
X_train_cvlog = cvec.fit_transform(X_train)
X_test_cvlog = cvec.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['\n', 'весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


In [602]:
logreg = LogisticRegression(C=0.1,
                          penalty='l2',
                          max_iter=1000)

In [603]:
logreg.fit(X_train_cvlog, y_train)
preds_cvlog = logreg.predict(X_test_cvlog)

TfidfVectorizer + NaiveBayes

In [604]:
tfidf = TfidfVectorizer(tokenizer=custom_tokenizer,
                        token_pattern=None,
                        stop_words=russian_stopwords,
                        lowercase=True,
                        max_features=1000,
                        ngram_range=(1, 1),
                        max_df=0.4,
                        min_df=5)

In [605]:
X_train_mnbtf = tfidf.fit_transform(X_train)
X_test_mnbtf = tfidf.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['\n', 'весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


In [606]:
mnb = MultinomialNB(alpha=0.1,
                    force_alpha=False,
                    fit_prior=False)

In [607]:
mnb.fit(X_train_mnbtf, y_train)
preds_mnbtf = mnb.predict(X_test_mnbtf)

### Сравнение результатов

In [608]:
print(classification_report(y_test, preds_cvlog))

              precision    recall  f1-score   support

         0.0       0.84      0.94      0.89      2906
         1.0       0.85      0.62      0.72      1418

    accuracy                           0.84      4324
   macro avg       0.84      0.78      0.80      4324
weighted avg       0.84      0.84      0.83      4324



In [609]:
print(classification_report(y_test, preds_mnbtf))

              precision    recall  f1-score   support

         0.0       0.85      0.86      0.86      2906
         1.0       0.71      0.69      0.70      1418

    accuracy                           0.81      4324
   macro avg       0.78      0.78      0.78      4324
weighted avg       0.80      0.81      0.81      4324



Naive Bayes в целом довольно мощный инструмент для текстовой классификации, и он показывает вполне хороший результат, но его все же лучше использовать для мультиклассовой разметки, в то время как логистическая регрессия лучший инструмент для 2 классов.

### Сравнение наиболее токсичных текстов

In [610]:
proba_cvlog = logreg.predict_proba(X_test_cvlog)

In [611]:
def get_most_toxic(x, y):
  merged_list = [(x[i], y[i][1]) for i in range(0, len(x))]
  sorted_list = sorted(merged_list, key=lambda x: x[1])
  return sorted_list[-10:]

In [612]:
 log_toxic = get_most_toxic(X_test, proba_cvlog)
 log_toxic

[('моча сюда не заходит И как привлекать их внимание? Флешмоб организовать что-ли? Если будешь надоедать моче борьбой с поехавшим, то сам будешь не сильно отличаться от поехавшего. Спейсачерам просто пора научиться игнорировать этого идиота и просто бампать другие треды, вместо того чтобы отвечать шизику. Всё равно никто не хочет с ним говорить, но у него реально в голове иллюзия создается, что он какой-то интересный человек, хотя по сути ему на лицо ссут, а он думает что с ним нормально общаются. Я думаю с ним в жизни никто не общается и даже когда ему на лицо ссут, банят, посылают нахуй, он думает что это общение и радуется как ребенок.\n',
  0.9999984415091223),
 ('Пошла нахуй свинья. Хохла на штык! Хороший хохол - дохлый хохол. Убей хохла, убей!\n',
  0.9999987435280309),
 ('лахтадырые и ольгинцы (Лахта, Ольгино) это которые провластные комменты пишут, мол у пендосов еще хуже, в европе пидоры. В рунете дохуя еще киберсотенцев - это хохляцкие ципсо, у них несколько этих самых ципсо,

In [613]:
proba_mnbtf = mnb.predict_proba(X_test_mnbtf)

In [614]:
 mnbtf_toxic = get_most_toxic(X_test, proba_mnbtf)
 mnbtf_toxic

[('Двачую этого. Свиньи уже в наглую пиздят что их тут нет и что они не срут в по мойке\n',
  0.9985533434410686),
 ('Сука, как же я обосрался с этого шерлока, до слёз блять\n',
  0.9986211864294432),
 ('мне поебать на его финансы и кто он. Не пизди, шлюха. Выкладывай своё ебло, мы поржем.\n',
  0.9986585288817517),
 ('Блядь, хохлы, вы бы хоть с проксей писали, няши тупые.\n',
  0.9987527126280894),
 ('Все, едем в пендостан ебать тупых пендосских шлюх.\n', 0.9987745152706767),
 ('БЕЛАРУСЬ, БЛЯТЬ, БЕЛАРУСЬ. СПИДОРСВИН, БЛЯТЬ. НЕПРОБИВАЕМАЯ ХОХЛИНА, СУКА. Какие-же хохлы дененераты, пиздец просто.\n',
  0.9989900493239645),
 ('Ты охуела, мразь? Я же тебя найду блять. Гридин не свинья, ты понял, петух? НЕ СМЕЙ НАЗЫВАТЬ КУЗЬМУ СВИНЬЕЙ ДЭБИЛ ГОРОХОВЫЙ!\n',
  0.9992794731901933),
 ('Пошла нахуй свинья. Хохла на штык! Хороший хохол - дохлый хохол. Убей хохла, убей!\n',
  0.9994918125204185),
 ('Пидорашек выселить, а не хохлов Ты сам себе противоречишь, дурачок.\n',
  0.9996357391070707),
 ('Ка

Пересечений текстов нет (неудивительно, тк логистическая регрессия поставила многим комментариям 1.0), в обоих случаях тексты токсичные, однако наиболее токсичные тексты нашел Naive Bayes.

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов.

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [615]:
import re
def custom_tokenizer(smth):
  smth = re.sub(r'[^\w\s]+|[\d]+', r'',smth)
  smth = smth.translate(str.maketrans('', '', string.punctuation))
  tokens = list(mystem.lemmatize(smth))
  tokens = [i for i in tokens if i not in {' ', '\n', '  '}]
  return tokens

In [616]:
X_train, X_test, y_train, y_test = train_test_split(data.comment.tolist(), data.toxic.tolist(), test_size=0.3, random_state=42)
# X_train_for_cvec = [custom_tokenizer(i) for i in X_train]
# X_test_for_cvec = [custom_tokenizer(i) for i in X_test]

In [617]:
cvec = CountVectorizer(tokenizer=custom_tokenizer,
                       token_pattern=None,
                       stop_words=russian_stopwords,
                       lowercase=True,
                       ngram_range=(1, 2),
                       max_df=1.0,
                       min_df=0.001
                       )

In [618]:
X_train_cv = cvec.fit_transform(X_train)
X_test_cv = cvec.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


In [619]:
classifiers = {'MultinomialNB': MultinomialNB(alpha=0.1),
               'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=10000,
                                                                criterion="log_loss",
                                                                class_weight='balanced'),
               'LogisticRegression': LogisticRegression(C=0.1,
                                                        penalty='l2',
                                                        max_iter=1000),
               'RandomForestClassifier': RandomForestClassifier(n_estimators=1000,
                                                                max_depth=10000)}

In [620]:
from sklearn.metrics import f1_score
import numpy as np

def get_toxic(k, v):
  if k == 'MultinomialNB':
    feature_probas = v.feature_log_prob_[1]
  if k == 'DecisionTreeClassifier':
    feature_probas = v.feature_importances_
  elif k == 'RandomForestClassifier':
    feature_probas = v.feature_importances_
  elif k == 'LogisticRegression':
    feature_probas = v.coef_[0]
  return feature_probas

In [621]:
def get_most_toxic(feature_probs, cvec):
  feature_names = cvec.get_feature_names_out()
  feature_probs = list(feature_probs)
  merged_list = [(feature_names[i], feature_probs[i]) for i in range(0, len(feature_names))]
  sorted_list = sorted(merged_list, key=lambda x: x[1])
  return sorted_list[-5:]

In [622]:
for k, v in classifiers.items():
  model = v
  model.fit(X_train_cv, y_train)
  y_pred_v = model.predict(X_test_cv)
  f1 = f1_score(y_test, y_pred_v)
  most_toxic = get_most_toxic(get_toxic(k, v), cvec)
  print(f'\n{k} : {f1},')
  [print(i) for i in most_toxic]


MultinomialNB : 0.7487609607319863,
('человек', -4.7369706089170265)
('просто', -4.713449336386332)
('свой', -4.461360314728199)
('весь', -4.3748760263393045)
('это', -3.5659423398971084)

DecisionTreeClassifier : 0.6487883683360258,
('это', 0.011159094390809959)
('тупой', 0.011390086356310268)
('твой', 0.011922345591039751)
('год', 0.016076487140025245)
('хохол', 0.020351661231020845)

LogisticRegression : 0.6712983065566652,
('русский', 1.1157974242372881)
('тупой', 1.2101488966295366)
('дебил', 1.2505350070804986)
('хохлов', 1.2515009698235882)
('хохол', 1.7926202871422294)

RandomForestClassifier : 0.7142857142857143,
('тупой', 0.009057990572498304)
('хохлов', 0.009560001027919447)
('год', 0.01054014500100382)
('русский', 0.010568879271957045)
('хохол', 0.018615758810921028)


Перепробовала параметры, которые были ранее, почему-то все модели стали показывать результаты хуже...